In [1]:
#import dependencies
import pandas as pd
import os

# you are here
os.getcwd()

'C:\\Users\\Alec\\Documents\\Programming Resources\\Boot Camp\\Pandas & Jupyter Notebook\\pandas-challenge'

## Preliminary Clean-up / Sampling

In [2]:
# designate filepath as variable - use variable for easy recall
infile_01 = ('resources\\purchase_data.csv')
purchase_data_df = pd.read_csv(infile_01)
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#obtain columns to list before really diving in as best practice 

root_column_list = purchase_data_df.columns.values.tolist()
root_column_list

['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price']

In [4]:
purchase_data_df.describe()

,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,91.755128,3.050987
std,225.310896,6.659444,52.697702,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,47.750000,1.980000
50%,389.500000,22.000000,92.000000,3.150000
75%,584.250000,25.000000,138.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


In [5]:
# Test for Null Values
purchase_data_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [6]:
NaN_test_df = purchase_data_df.dropna(how='any')
NaN_test_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [7]:
no_dupe_purchase_data_df = purchase_data_df.drop_duplicates()

# Player Count

In [8]:
# isolate unique value of SN series, double check w/print, move to dict for database creation

SN_list = purchase_data_df['SN'].unique()
unique_player_count = len(SN_list)

#print(unique_player_count)

# build df
unique_players = {
    'Total Players': [unique_player_count],
}

total_players_df = pd.DataFrame(unique_players)
total_players_df

,Total Players
0,576


# Purchasing Analysis (Total)

In [9]:
# ------------------------Number of Unique Items------------------------ #

unique_itemID_list = purchase_data_df['Item ID'].unique()
unique_item_num = int(len(unique_itemID_list))

# ------------------------Average purchase price------------------------ #

sum_prices = purchase_data_df['Price'].sum()

num_purchases = purchase_data_df['Purchase ID'].count()

avg_purchase_price = sum_prices / num_purchases

# ------------------------Summary DataFrame------------------------ #

purchase_analysis_df = pd.DataFrame({
    'Number of Unique Items': [unique_item_num],
    'Average Price': [avg_purchase_price],
    'Number of Purchases': [num_purchases],
    'Total Revenue': [sum_prices],
})

# format the data values
purchase_analysis_df = purchase_analysis_df.round(2)
purchase_analysis_df['Average Price'] = purchase_analysis_df['Average Price'].map('${:,.2f}'.format)
purchase_analysis_df['Number of Purchases'] = purchase_analysis_df['Number of Purchases'].map('{:,}'.format)
purchase_analysis_df['Total Revenue'] = purchase_analysis_df['Total Revenue'].map('${:,.2f}'.format)

# display the dataframe of results
purchase_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


# Gender Demographics

In [10]:
# ------------------------ Gender Counts ------------------------ #
# obtain the info needed
user_demo = purchase_data_df.loc[:, ['Gender', 'SN', 'Age']]
# remove duplicates --> usinge .unique will not work for this because we don't want an array for calculations
user_demo = user_demo.drop_duplicates()
# build new df
user_demo_df = pd.DataFrame(user_demo)

# isolate series with info needed - peroforming calclulations across the new df allows us to avoid having to obtain
# individual variables then perform math on those variables.

gender_counts = user_demo_df['Gender'].value_counts()

# calculate % by dividing individual by whole across whole df
gender_prcnt = gender_counts / unique_player_count * 100

# Build the Summary df
gender_demo_df = pd.DataFrame({
    'Totals': gender_counts,
    'Percentage of Players': gender_prcnt
})

# format the data
gender_demo_df['Percentage of Players'] = gender_demo_df['Percentage of Players'].map('{:,.2f}%'.format)


gender_demo_df

,Totals,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


# Purchasing Analysis (Gender)

In [11]:
# Find out which gender spends more? indication of 
# behavior \ lack of patience \ desire for reward\gratification?

# --> .rename() following .mean, .sum, count functions to avoid error in which the entire result is dropped into
# data dict in a way that cannot be utilized by the program

# retrieve the information needed, assign to df for later use
gender_purchase_df = purchase_data_df.loc[:, ['Purchase ID', 'Gender', 'Price']]

# obtain the counts per gender by grouping the count of purchases per gender
gender_purchase_counts = gender_purchase_df.groupby(['Gender']).count()['Price'].rename('Purchase Count')

# obtain the sum of money spent per gender by grouping gender and using .sum() on price series
gender_purchase_total = gender_purchase_df.groupby(['Gender']).sum()['Price'].rename('Total Purchase Value')

# obtain the average grouping the price spent per gender and using the .mean() function
gender_avg = gender_purchase_df.groupby(['Gender']).mean()['Price'].rename('Average Purchase Price')

# isolate the total spent per gender (gender_purchase_total), divide by the number of each gender

gender_purchase_avg = gender_purchase_total / gender_counts

# build the dataframe
gender_purchase_demo_df = pd.DataFrame({
    'Purchase Count': gender_purchase_counts,
    'Average Purchase Price': gender_avg,
    'Total Purchase Value': gender_purchase_total,
    'Avg Total Purchase per Person':gender_purchase_avg,

})

# format the dataframe
gender_purchase_demo_df['Purchase Count'] = gender_purchase_demo_df['Purchase Count'].map('{:,.2f}'.format)
gender_purchase_demo_df['Average Purchase Price'] = gender_purchase_demo_df['Average Purchase Price'].map('${:,.2f}'.format)
gender_purchase_demo_df['Total Purchase Value'] = gender_purchase_demo_df['Total Purchase Value'].map('${:,.2f}'.format)
gender_purchase_demo_df['Avg Total Purchase per Person'] = gender_purchase_demo_df['Avg Total Purchase per Person'].map('${:,.2f}'.format)

gender_purchase_demo_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113.00,$3.20,$361.94,$4.47
Male,652.00,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15.00,$3.35,$50.19,$4.56


# Age Demographics

In [12]:
# Estabilish the bins
age_bin = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9 ,39.9, 1000]
group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

# add new category (Age Ranges) for existing players
purchase_data_df['Age Ranges'] = pd.cut(purchase_data_df['Age'], age_bin, labels=group_names)

# isolate Values so we only have what we need
age_demo_df = purchase_data_df[['Gender','SN', 'Age','Age Ranges']]
# we dont need duplicate values
age_demo_df = age_demo_df.drop_duplicates()

# define the totals by taking the value count of each new age range
age_totals = age_demo_df['Age Ranges'].value_counts()

# percent = total / whole * 100
age_percent = age_totals / unique_player_count *100

# build the dataframe
age_df = pd.DataFrame({
    'Total Count': age_totals,
    'Percentage of Players': age_percent,
})


# sort the index for readability
age_df = age_df.sort_index(ascending = True)

age_df['Total Count'] = age_df['Total Count'].map('{:,}'.format)
age_df['Percentage of Players'] = age_df['Percentage of Players'].map('{:,.2f}%'.format)


age_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


# Purchasing Analysis (Age)

In [13]:
# grab the age info binned from above
purchase_data_df['Age Ranges'] = pd.cut(purchase_data_df['Age'], age_bin, labels=group_names)

age_range_df = pd.cut(purchase_data_df['Age'], age_bin, labels=group_names)

# gather data as above with Gender Demographics except we are comparing the amounts spent vs the Age Ranges of the root data frame
age_range_totals = purchase_data_df.groupby(['Age Ranges']).count()['Price'].rename('Purchase Count')
age_avg_purchase = purchase_data_df.groupby(['Age Ranges']).mean()['Price'].rename('Average Purchase Price')
age_purchase_total = purchase_data_df.groupby(['Age Ranges']).sum()['Price'].rename('Total Purchase Value')

# isolate the Age Range & SN
SN_age_range_df = purchase_data_df[['Age Ranges', 'SN']]

#remove duplicates so we have per person data, not per transaction data --> (age_range_totals)
unique_SN_age_range_df = SN_age_range_df.drop_duplicates()

#obtain the counts to use in arithmatic
unique_SN_age_range = unique_SN_age_range_df['Age Ranges'].value_counts()

# execute arithmatic and solve for average purchase per unique user in age range
avg_purchase_per_age_range = age_purchase_total / unique_SN_age_range

age_analysis_df = pd.DataFrame({
    'Purchase Count': age_range_totals,
    'Average Purchase Price': age_avg_purchase,
    'Total Purchase Value': age_purchase_total,
    'Avg Total Purchase per Person': avg_purchase_per_age_range,
    'Age Ranges': group_names,

})

# set index for info clarity - sort
age_analysis_df = age_analysis_df.set_index('Age Ranges')
age_analysis_df = age_analysis_df.sort_index(axis=1)


# format the dataframe

age_analysis_df['Purchase Count'] = age_analysis_df['Purchase Count'].map('{:,}'.format)
age_analysis_df['Average Purchase Price'] = age_analysis_df['Average Purchase Price'].map('${:,.2f}'.format)
age_analysis_df['Total Purchase Value'] = age_analysis_df['Total Purchase Value'].map('${:,.2f}'.format)
age_analysis_df['Avg Total Purchase per Person'] = age_analysis_df['Avg Total Purchase per Person'].map('${:,.2f}'.format)



age_analysis_df

,Average Purchase Price,Avg Total Purchase per Person,Purchase Count,Total Purchase Value
Age Ranges,,,,
<10,$3.35,$4.54,23,$77.13
10-14,$2.96,$3.76,28,$82.78
15-19,$3.04,$3.86,136,$412.89
20-24,$3.05,$4.32,365,"$1,114.06"
25-29,$2.90,$3.81,101,$293.00
30-34,$2.93,$4.12,73,$214.00
35-39,$3.60,$4.76,41,$147.67
40+,$2.94,$3.19,13,$38.24


# Top Spenders

In [14]:
# Gather information as above - we just need to compare data that is already present in main df
# so there's no real need to make a new df
# begin arithmatic
top_spender_total = purchase_data_df.groupby(['SN']).sum()['Price'].rename('Total Purchase Value')
top_spender_count = purchase_data_df.groupby(['SN']).count()['Price'].rename('Purchase Count')
top_spender_avg = purchase_data_df.groupby(['SN']).mean()['Price'].rename('Average Purchase Price')

#build df from functions
top_spender_df = pd.DataFrame({
    'Purchase Count': top_spender_count,
    'Average Purchase Price': top_spender_avg,
    'Total Purchase Value': top_spender_total
})


# format the dataframe

top_spender_df['Purchase Count'] = top_spender_df['Purchase Count'].map('{:,}'.format)
top_spender_df['Average Purchase Price'] = top_spender_df['Average Purchase Price'].map('${:,.2f}'.format)
top_spender_df['Total Purchase Value'] = top_spender_df['Total Purchase Value'].map('${:,.2f}'.format)


# sort by total spent (ascending = False) to see the highest to lowest spenders
# show the top 5 through the use of the .head() function
top_5_spender_df = top_spender_df.sort_values('Total Purchase Value', ascending = False).head(5)
top_5_spender_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Haillyrgue51,3,$3.17,$9.50
Phistym51,2,$4.75,$9.50
Lamil79,2,$4.64,$9.29
Aina42,3,$3.07,$9.22
Saesrideu94,2,$4.59,$9.18


# Most Popular Items

In [15]:
# gather data needed for Item Comparisons - make into df
item_data = purchase_data_df.loc[:, ['Item ID', 'Item Name', 'Price']]
item_data

# begin arithmatic to find the money made via purchase of a particular item
item_purchase_total = item_data.groupby(['Item ID','Item Name']).sum()['Price'].rename('Total Purchase Value')

# average price of the item - it seems this game offers dynamic pricing --> note issues with that in analysis
item_avg_price = item_data.groupby(['Item ID','Item Name']).mean()['Price'].rename('Average Item Price')

# total count of items purchase per Item ID
item_count = item_data.groupby(['Item ID','Item Name']).count()['Price'].rename('Purchase Count')

# build the summary dataframe
item_data_df = pd.DataFrame({
    'Purchase Count': item_count,
    'Average Item Price': item_avg_price,
    'Total Purchase Value': item_purchase_total,    
})

# format the dataframe

item_data_df['Purchase Count'] = item_data_df['Purchase Count'].map('{:,}'.format)
item_data_df['Average Item Price'] = item_data_df['Average Item Price'].map('${:,.2f}'.format)
item_data_df['Total Purchase Value'] = item_data_df['Total Purchase Value'].map('${:,.2f}'.format)



# Limit to the top 5 for quick reference
top_5_item_data = item_data_df.sort_values('Purchase Count', ascending=False).head(5)
top_5_item_data

,,Purchase Count,Average Item Price,Total Purchase Value
Item ID,Item Name,,,
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
60,Wolf,8,$3.54,$28.32


# Most Profitable Items

In [16]:
# now we just need the items with highest Purchase Value
# so we can use the already made table and sort via purchase count
# to display which are the most used by players.

# format the dataframe

item_data_df.sort_values('Total Purchase Value', ascending=False).head(5)

,,Purchase Count,Average Item Price,Total Purchase Value
Item ID,Item Name,,,
63,Stormfury Mace,2,$4.99,$9.98
29,"Chaos, Ender of the End",5,$1.98,$9.90
173,Stormfury Longsword,2,$4.93,$9.86
38,"The Void, Vengeance of Dark Magic",4,$2.37,$9.48
143,Frenzied Scimitar,6,$1.56,$9.36
